### Load libraries

In [ ]:
# import matplotlib.pyplot as plt
import pathlib
import mne
from mne.preprocessing import create_ecg_epochs, create_eog_epochs
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch
from mne.channels.montage import get_builtin_montages
from mne.io import read_raw_bdf, concatenate_raws
import os
import numpy as np
from numpy import savetxt
from numpy import loadtxt

import matplotlib
matplotlib.use('Qt5Agg')

### Load data

In [ ]:
data_folder = '/Users/christinadelta/Desktop/beads_pilot/beads'
raw_file = os.path.join(data_folder, 'sub_test_01.bdf')

raw1 = read_raw_bdf(raw_file, eog=['EXG3', 'EXG4','EXG5', 'EXG6'], stim_channel='auto', 
                    exclude=(['EXG7', 'EXG8', 'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp']), 
                    preload=True)

raw2_file = os.path.join(data_folder, 'sub_test_01_block_02.bdf')
raw2 = read_raw_bdf(raw2_file, eog=['EXG3', 'EXG4','EXG5', 'EXG6'], stim_channel='auto', exclude=(['EXG7', 'EXG8']),
                    preload=True)

raw3_file = os.path.join(data_folder, 'sub_test_01_block_03.bdf')
raw3 = read_raw_bdf(raw3_file, eog=['EXG3', 'EXG4','EXG5', 'EXG6'], stim_channel='auto', exclude=(['EXG7', 'EXG8']),
                    preload=True)

raw4_file = os.path.join(data_folder, 'sub_test_01_block_04.bdf')
raw4 = read_raw_bdf(raw4_file, eog=['EXG3', 'EXG4','EXG5', 'EXG6'], stim_channel='auto', exclude=(['EXG7', 'EXG8']),
                    preload=True)

### Concatinate the different raw files

In [ ]:
raws = [] # create empty list to store the raws

raws.append(raw1)
raws.append(raw2)
raws.append(raw3)
raws.append(raw4)
raws

# concatinate
raw = concatenate_raws(raws)

In [ ]:
raw.plot()

### Load the events

In [ ]:
events = mne.find_events(raw)

events1 = events[0:294]
events1.shape

events2 = events[295:550]
events2.shape

events3 = events[550:815]
events3.shape

events4 = events[815:1051]
events4.shape

In [ ]:
events_block1 = loadtxt('newevents.csv', delimiter=',')
events_block1 = np.int8(events_block1)
events_block1.shape

events_block2 = loadtxt('events_block2.csv', delimiter=',')
events_block2 = np.int8(events_block2)
events_block2.shape

events_block3 = loadtxt('events_block3.csv', delimiter=',')
events_block3 = np.int8(events_block3)
events_block3.shape

events_block4 = loadtxt('events_block4.csv', delimiter=',')
events_block4 = np.int8(events_block4)
events_block4.shape

events[0:294,2] = events_block1
events[295:550,2] = events_block2
events[550:815,2] = events_block3
events[815:1051,2] = events_block4

events.shape

In [ ]:
event_id = {
    "easy/blue_urn": 1,
    "easy/green_urn": 2,
    "diff/blue_urn": 3,
    "diff/green_urn": 4,
    "resprp": 5,
    "bu": 6,
    "gu": 7,
    "draw": 8,
    "conf_scrn": 9,
    "rate_1": 10,
    "feed_1": 14,
    "feed_2": 15,
    "cond1": 100,
    "cond2": 101,
    "start": 102,
    "end": 103,
}

### Filter data, re-reference and run ICA

In [ ]:
filtered = raw.copy().filter(l_freq=1, h_freq=20) 

filtered.plot(title='filtered')
#filtered2.plot(title='filtered2')

In [ ]:
# re-reference
ref_channels = ['EXG1', 'EXG2']
filtered.set_eeg_reference(ref_channels)

### Add montage (Biosemi64) for topomaps

In [ ]:
biosemi_montage = mne.channels.make_standard_montage('biosemi64')
filtered.set_channel_types({'EXG1': 'misc', 'EXG2': 'misc'})
filtered_montage = filtered.copy().set_montage(biosemi_montage, on_missing='warn')
filtered_montage.plot()

In [ ]:
#ica = mne.preprocessing.ICA(n_components=20, random_state=0)
#ica.fit(filtered_montage)
#filtered_montage.load_data()
# ica.plot_sources(filtered, show_scrollbars=False)
ica.plot_components()

In [ ]:
ica.plot_sources(filtered_montage, show_scrollbars=True)

In [ ]:
# try the find_bads_eog() algorithm 
bad_idx, scores = ica.find_bads_eog(filtered_montage, ch_name=(['EXG3','EXG4','EXG5','EXG6']), threshold=2)
print(bad_idx)

In [ ]:
# ica.exclude = [0, 1, 6, 8, 9, 10] 
ica.exclude = bad_idx 
ica.apply(filtered_montage, exclude=ica.exclude).plot(events = events, event_id = event_id)

### Epoch the data

In [ ]:
stimulus_epochs = mne.Epochs(filtered_montage, events, event_id=event_id, tmin=-0.3, tmax=0.8, preload=True)
fig = stimulus_epochs.plot(events = events, event_id = event_id)

In [ ]:
# Downsample to 100 Hz
print('Original sampling rate:', stimulus_epochs.info['sfreq'], 'Hz')
epochs_resampled = stimulus_epochs.resample(500, npad="auto")
print('New sampling rate:', epochs_resampled.info['sfreq'], 'Hz')

### Look at Average ERPs

In [ ]:
stimulus_epochs.info['sfreq']

In [ ]:
diff_epochs = epochs_resampled['diff'].average()
easy_epochs = epochs_resampled['easy'].average()

In [ ]:
fig1 = diff_epochs.plot()
fig2 = easy_epochs.plot(spatial_colors=True)

In [ ]:
frontal_total = ['F1', 'F3', 'F5', 'F7', 'F2', 'F4', 'F6', 'F8', 'Fz']
parietal_total = ['P1', 'P3', 'P5', 'P7', 'P2', 'P4', 'P6', 'P8', 'P10']

In [ ]:
evokeds = dict(diff_frontal=diff_epochs, easy_frontal=easy_epochs)
picks = [f'F{n}' for n in range(1,10)]
mne.viz.plot_compare_evokeds(evokeds, picks=picks, combine='mean')

In [ ]:
evokeds = dict(diff_parietal=diff_epochs, easy_parietal=easy_epochs)
picks = [f'P{n}' for n in range(1,10)]
mne.viz.plot_compare_evokeds(evokeds, picks=picks, combine='mean')

In [ ]:
# look at topomaps
# easy_epochs.plot_topomap(times=[-0.2, 0.2, 0.4, 0.6], average=0.05)
diff_epochs.plot_topomap(times=[-0.2, 0.2, 0.4, 0.6], average=0.05)

In [ ]:
# add joint topomaps
# easy_epochs.plot_joint()
diff_epochs.plot_joint()

In [ ]:
# compare conditions
diff_minus_easy = mne.combine_evoked([easy_epochs, diff_epochs], weights=[1, -1])
diff_minus_easy.plot_joint()

### Time-Frequency Analysis

In [ ]:
epochs_resampled.info

In [ ]:
# pick epochs 
difficult = epochs_resampled['diff']
difficult.plot_psd(fmin=2., fmax=40., average=True, bandwidth=2)

In [ ]:
# define frequencies of interest (log-spaced)
freqs = np.logspace(*np.log10([2, 40]), num=20)
freqs

In [ ]:
# run tf analysis. Also reurn itc (intertrial coherence)
n_cycles = freqs / 2.  # different number of cycle per frequency
power, itc = mne.time_frequency.tfr_morlet(difficult, freqs=freqs, n_cycles=n_cycles, use_fft=True,
                                           return_itc=True, decim=3, n_jobs=1)

In [ ]:
power.crop(-0.1, 0.7)  # crop to remove edge artifacts
itc.crop(-0.1, 0.7)  # crop to remove edge artifacts

### Inspect power

In [ ]:
# specify baseline period
baseline_mode = 'logratio'
baseline = (None, 0)

### Plot power topomaps

In [ ]:
(power.copy()
 .pick_types(eeg=True, meg=False)
 .plot_topo())

### Look at specific channels

In [ ]:
power.plot(picks='P2', baseline=baseline, mode=baseline_mode)

### Plot topomaps for specified frequency ranges

In [ ]:
import matplotlib.pyplot as plt

fig, axis = plt.subplots(1, 3, figsize=(7, 4))
power.plot_topomap(ch_type='eeg', tmin=0.3, tmax=0.8, fmin=4, fmax=7,
                   baseline=baseline, mode=baseline_mode, axes=axis[0],
                   title='Theta', show=False, contours=1)

power.plot_topomap(ch_type='eeg', tmin=0.3, tmax=0.8, fmin=8, fmax=12,
                   baseline=baseline, mode=baseline_mode, axes=axis[1],
                   title='Alpha', show=False, contours=1)

power.plot_topomap(ch_type='eeg', tmin=0.3, tmax=1., fmin=15, fmax=30,
                   baseline=baseline, mode=baseline_mode, axes=axis[2],
                   title='Beta', show=False, contours=1)

mne.viz.tight_layout()
plt.show()

### Joint TF Plot

In [ ]:
power.plot_joint(baseline=baseline, mode='mean', tmin=None, tmax=None,
                 timefreqs=[(0.3, 5), (0.3, 15.)])
plt.show()


### Inspect ITC

In [ ]:
itc.plot_topo(title='Inter-Trial coherence', vmin=0., vmax=0.5, cmap='Reds')